In [1]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(harmony))
suppressMessages(library(Nebulosa))
suppressMessages(library(ggpubr))
suppressMessages(library(Ipaper))
suppressMessages(library(phateR))
suppressMessages(library(viridis))
suppressMessages(library(SeuratDisk))
suppressMessages(library(SingleCellExperiment))
suppressMessages(library(destiny))
suppressMessages(library(plotly))

In [2]:
obj.human <- readRDS("../../../IntegrativeAnalysis/Fibroblast/data/coembed/coembed.annotation.Rds")

In [ ]:
obj.human

In [ ]:
## only include Myofib, SCARA5+ and Fib_0, as they are also recovered by mouse data
df <- obj.human@meta.data %>%
    as.data.frame() %>%
    subset(., annotation %in% c("Fib_SCARA5", "Myofib", "Fib_0"))

obj.human <- obj.human[, rownames(df)]

In [ ]:
obj.human

In [ ]:
cols <- ArchR::paletteDiscrete(unique(obj.human@meta.data[, "annotation"]))
cols

In [ ]:
source("../../utils/trajectory_ArchR.R")
traj1 <- c("Fib_SCARA5", "Myofib")
traj2 <- c("Fib_0", "Myofib")

In [ ]:
## umap
obj.human <- RunUMAP(obj.human, 
               dims = 1:50, 
               reduction = 'harmony',
               reduction.name = "umap_harmony_v2",
               reduction.ke = 'umap_harmony_v2_',
              verbose = FALSE,
                    min.dist = 0.4)

In [ ]:
obj.human <- addTrajectoryA(obj.human, reducedDims = "umap_harmony_v2", 
                          name="SCARA5_Myofib_umap", trajectory = traj1,
                         groupBy = "annotation",
                            useAll = FALSE)

obj.human <- addTrajectoryA(obj.human, reducedDims = "umap_harmony_v2", 
                          name="Fib_0_Myofib_umap", trajectory = traj2,
                         groupBy = "annotation",
                            useAll = FALSE)

p1 <- DimPlot(obj.human, reduction = "umap_harmony_v2", cols = cols) +
    xlab("UMAP1") + ylab("UMAP2") +
    theme(axis.ticks = element_blank(),
         axis.text = element_blank())


p2 <- plotTrajectoryA(obj.human, trajectory = "SCARA5_Myofib_umap", 
                      name = "SCARA5_Myofib_umap",
                      embedding = "umap_harmony_v2", size = 1, plotAs = "points",
                     addArrow = FALSE) +
   xlab("UMAP 1") + ylab("UMAP 2") +
   theme_cowplot() + ggtitle("SCARA5_Myofib_umap")


p3 <- plotTrajectoryA(obj.human, trajectory = "Fib_0_Myofib_umap", 
                      name = "Fib_0_Myofib_umap",
                      embedding = "umap_harmony_v2", size = 1, plotAs = "points",
                     addArrow = FALSE) +
   xlab("UMAP 1") + ylab("UMAP 2") +
   theme_cowplot() + ggtitle("Fib_0_Myofib_umap")

options(repr.plot.height = 5, repr.plot.width = 15)

p1 + p2 + p3

In [ ]:
df1 <- obj.human@meta.data %>%
    as.data.frame() %>%
    subset(., !is.na(SCARA5_Myofib_umap)) %>%
    group_by(patient_group) %>%
    mutate(time_point = cut(SCARA5_Myofib_umap, 
                            breaks = seq(0, 100, 5), 
                            labels = seq(1, 20, 1))) %>%
    ungroup() %>%
    group_by(patient_group, time_point) %>%
    summarise(num_cells = n()) %>%
    mutate(cell_proportion = num_cells / sum(num_cells))

df2 <- obj.human@meta.data %>%
    as.data.frame() %>%
    subset(., !is.na(Fib_0_Myofib_umap)) %>%
    group_by(patient_group) %>%
    mutate(time_point = cut(Fib_0_Myofib_umap, 
                            breaks = seq(0, 100, 5), 
                            labels = seq(1, 20, 1))) %>%
    ungroup() %>%
    group_by(patient_group, time_point) %>%
    summarise(num_cells = n()) %>%
    mutate(cell_proportion = num_cells / sum(num_cells))

df1$time_point <- 5*as.numeric(df1$time_point)
df2$time_point <- 5*as.numeric(df2$time_point)

p1 <- ggplot(data = df1, aes(x = time_point, y = cell_proportion,
                                color = patient_group)) +
        geom_smooth(se = FALSE) + ggtitle("UMAP: SCARA5 -> Myofib") +
    xlab("Pseudo-time") + ylab("Cell proportion") +
    theme_cowplot() +
        theme(legend.title = element_blank())

p2 <- ggplot(data = df2, aes(x = time_point, y = cell_proportion,
                                color = patient_group)) +
        geom_smooth(se = FALSE) + ggtitle("UMAP: Fib0 -> Myofib") +
    xlab("Pseudo-time") + ylab("Cell proportion") +
    theme_cowplot() +
        theme(legend.title = element_blank())

options(repr.plot.height = 4, repr.plot.width = 12)
p1 + p2

In [ ]:
## duffusion map
matDR <- obj.human@reductions$harmony@cell.embeddings

dm <- DiffusionMap(as.matrix(matDR),
                   verbose = TRUE)

In [ ]:
embedding <- as.data.frame(dm)[, c("DC1", "DC2", "DC3")]
colnames(embedding) <- paste0("DC_", 1:ncol(embedding))

obj.human[['dm']] <- CreateDimReducObject(embeddings = as.matrix(embedding), 
                                              key = "DC")

In [ ]:
obj.human <- addTrajectoryA(obj.human, reducedDims = "dm", 
                          name="SCARA5_Myofib_dm", trajectory = traj1,
                         groupBy = "annotation", 
                            useAll = FALSE)

obj.human <- addTrajectoryA(obj.human, reducedDims = "dm", 
                          name="Fib_0_Myofib_dm", trajectory = traj2,
                         groupBy = "annotation", 
                            useAll = FALSE)


p1 <- DimPlot(obj.human, reduction = "dm", cols = cols) +
    xlab("DC 1") + ylab("DC 2") +
    theme(axis.ticks = element_blank(),
         axis.text = element_blank())


p2 <- plotTrajectoryA(obj.human, trajectory = "SCARA5_Myofib_dm", 
                      name = "SCARA5_Myofib_dm",
                      embedding = "dm", size = 1, plotAs = "points",
                     addArrow = FALSE) +
   xlab("DC 1") + ylab("DC 2") +
   theme_cowplot() + ggtitle("SCARA5_Myofib_dm")


p3 <- plotTrajectoryA(obj.human, trajectory = "Fib_0_Myofib_dm", 
                      name = "Fib_0_Myofib_dm",
                      embedding = "dm", size = 1, plotAs = "points",
                     addArrow = FALSE) +
   xlab("DC 1") + ylab("DC 2") +
   theme_cowplot() + ggtitle("Fib_0_Myofib_umap")

options(repr.plot.height = 5, repr.plot.width = 15)

p1 + p2 + p3

In [ ]:
df1 <- obj.human@meta.data %>%
    as.data.frame() %>%
    subset(., !is.na(SCARA5_Myofib_dm)) %>%
    group_by(patient_group) %>%
    mutate(time_point = cut(SCARA5_Myofib_dm, 
                            breaks = seq(0, 100, 5), 
                            labels = seq(1, 20, 1))) %>%
    ungroup() %>%
    group_by(patient_group, time_point) %>%
    summarise(num_cells = n()) %>%
    mutate(cell_proportion = num_cells / sum(num_cells))

df2 <- obj.human@meta.data %>%
    as.data.frame() %>%
    subset(., !is.na(Fib_0_Myofib_dm)) %>%
    group_by(patient_group) %>%
    mutate(time_point = cut(Fib_0_Myofib_dm, 
                            breaks = seq(0, 100, 5), 
                            labels = seq(1, 20, 1))) %>%
    ungroup() %>%
    group_by(patient_group, time_point) %>%
    summarise(num_cells = n()) %>%
    mutate(cell_proportion = num_cells / sum(num_cells))


df1$time_point <- 5*as.numeric(df1$time_point)
df2$time_point <- 5*as.numeric(df2$time_point)

p1 <- ggplot(data = df1, aes(x = time_point, y = cell_proportion,
                                color = patient_group)) +
        geom_smooth(se = FALSE) + ggtitle("DM: SCARA5 -> Myofib") +
    xlab("Pseudo-time") + ylab("Cell proportion") +
    theme_cowplot() +
        theme(legend.title = element_blank())

p2 <- ggplot(data = df2, aes(x = time_point, y = cell_proportion,
                                color = patient_group)) +
        geom_smooth(se = FALSE) + ggtitle("DM: Fib0 -> Myofib") +
    xlab("Pseudo-time") + ylab("Cell proportion") +
    theme_cowplot() +
        theme(legend.title = element_blank())

options(repr.plot.height = 4, repr.plot.width = 12)
p1 + p2

In [ ]:
# # run PHATE
# phate_embedding <- phate(matDR, n.jobs = 20, npca = NULL)
# colnames(phate_embedding$embedding) <- c("PHATE_1", "PHATE_2")
# obj.human[['phate']] <- CreateDimReducObject(embeddings = phate_embedding$embedding)

In [ ]:
# obj.human <- addTrajectoryA(obj.human, reducedDims = "phate", 
#                           name="SCARA5_Myofib_phate", trajectory = traj1,
#                          groupBy = "annotation",
#                             useAll = TRUE)

# obj.human <- addTrajectoryA(obj.human, reducedDims = "phate", 
#                           name="Fib0_Myofib_phate", trajectory = traj2,
#                          groupBy = "annotation",
#                             useAll = TRUE)

In [ ]:
# p1 <- DimPlot(obj.human, reduction = "phate", cols = cols) +
#     xlab("PHATE 1") + ylab("PHATE 2") +
#     theme(axis.ticks = element_blank(),
#          axis.text = element_blank())

# p2 <- plotTrajectoryA(obj.human, trajectory = "SCARA5_Myofib_phate", 
#                       name = "SCARA5_Myofib_phate",
#                       embedding = "phate", size = 1, plotAs = "points",
#                      addArrow = FALSE) +
#    xlab("PHATE 1") + ylab("PHATE 2") +
#    theme_cowplot() + ggtitle("SCARA5_Myofib_phate")

# p3 <- plotTrajectoryA(obj.human, trajectory = "Fib0_Myofib_phate", 
#                       name = "Fib0_Myofib_phate",
#                       embedding = "phate", size = 1, plotAs = "points",
#                      addArrow = FALSE) +
#    xlab("PHATE 1") + ylab("PHATE 2") +
#    theme_cowplot() + ggtitle("Fib0_Myofib_phate")

# options(repr.plot.height = 5, repr.plot.width = 15)

# p1 + p2 + p3

In [ ]:
# df1 <- obj.human@meta.data %>%
#     as.data.frame() %>%
#     subset(., !is.na(SCARA5_Myofib_phate)) %>%
#     group_by(patient_group) %>%
#     mutate(time_point = cut(SCARA5_Myofib_phate, 
#                             breaks = seq(0, 100, 5), 
#                             labels = seq(1, 20, 1))) %>%
#     ungroup() %>%
#     group_by(patient_group, time_point) %>%
#     summarise(num_cells = n()) %>%
#     mutate(cell_proportion = num_cells / sum(num_cells))


# df2 <- obj.human@meta.data %>%
#     as.data.frame() %>%
#     subset(., !is.na(Fib0_Myofib_phate)) %>%
#     group_by(patient_group) %>%
#     mutate(time_point = cut(Fib0_Myofib_phate, 
#                             breaks = seq(0, 100, 5), 
#                             labels = seq(1, 20, 1))) %>%
#     ungroup() %>%
#     group_by(patient_group, time_point) %>%
#     summarise(num_cells = n()) %>%
#     mutate(cell_proportion = num_cells / sum(num_cells))

# df1$time_point <- 5*as.numeric(df1$time_point)
# df2$time_point <- 5*as.numeric(df2$time_point)

# p1 <- ggplot(data = df1, aes(x = time_point, y = cell_proportion,
#                                 color = patient_group)) +
#         geom_smooth(se = FALSE) + ggtitle("Phate: SCARA5 -> Myofib") +
#     xlab("Pseudo-time") + ylab("Cell proportion") +
#     theme_cowplot() +
#         theme(legend.title = element_blank())

# p2 <- ggplot(data = df2, aes(x = time_point, y = cell_proportion,
#                                 color = patient_group)) +
#         geom_smooth(se = FALSE) + ggtitle("Phate: Fib0 -> Myofib") +
#     xlab("Pseudo-time") + ylab("Cell proportion") +
#     theme_cowplot() +
#         theme(legend.title = element_blank())

# options(repr.plot.height = 4, repr.plot.width = 12)
# p1 + p2

In [ ]:
# # # Extract tSNE information from Seurat Object
# dc_1 <- obj.human[["dm"]]@cell.embeddings[,1]
# dc_2 <- obj.human[["dm"]]@cell.embeddings[,2]
# dc_3 <- obj.human[["dm"]]@cell.embeddings[,3]

# # Prepare a dataframe for cell plotting
# plot.data <- FetchData(object = obj.human, 
#                        vars = c("DC_1", "DC_2", "DC_3", "annotation"))

# # Make a column of row name identities (these will be your cell/barcode names)
# plot.data$label <- paste(rownames(plot.data))

# # Plot your data, in this example my Seurat object had 21 clusters (0-20)
# fig <- plot_ly(data = plot.data, 
#         x = ~DC_1, y = ~DC_2, z = ~DC_3, 
#         color = ~annotation, 
#         colors = cols,
#         type = "scatter3d", 
#         mode = "markers", 
#         marker = list(size = 5, width=2), # controls size of points
#         text=~annotation, #This is that extra column we made earlier for which we will use for cell ID
#         hoverinfo="text") #When you visualize your plotly object, hovering your mouse pointer over a point shows cell names

# options(repr.plot.height = 10, repr.plot.width = 10)

# fig

In [ ]:
saveRDS(obj.human,
       file = "../../../IntegrativeAnalysis/Fibroblast/data/coembed/coembed.annotation.trajectory.Rds")

In [ ]:
sessionInfo()